In [ ]:
from utils_load_data import load_cifar10, load_cifar10_img_form
import numpy as np
import pickle
import os
import datetime
from keras import layers
from keras import models
from keras import optimizers
from keras.utils import to_categorical

In [ ]:
def load_cifar10_batch(directory):
    """ load batch of cifar """
    with open(directory, 'rb') as fo:
        datadict = pickle.load(fo, encoding='bytes')
    X = np.array(datadict[b'data'])
    Y = np.array(datadict[b'labels'])
    return X, Y

def load_cifar10(directory):
    """ load all of cifar """
    train_data = []
    train_labels = []
    for b in range(1, 6):
        f = os.path.join(directory, 'data_batch_%d' % (b,))
        X, Y = load_cifar10_batch(f)
        train_data.append(X)
        train_labels.append(Y)
    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    del X, Y
    test_data, test_labels = load_cifar10_batch(os.path.join(directory, 'test_batch'))
    return train_data, train_labels, test_data, test_labels

def load_cifar10_img_form(directory):
    """ load all of cifar as image form """
    train_data, train_labels, test_data, test_labels = load_cifar10(directory)
    R, testR = train_data[:, :1024].reshape(-1, 32, 32, 1), test_data[:, :1024].reshape(-1, 32, 32, 1)
    G, testG = train_data[:, 1024:2048].reshape(-1, 32, 32, 1), test_data[:, 1024:2048].reshape(-1, 32, 32, 1)
    B, testB = train_data[:, 2048:].reshape(-1, 32, 32, 1), test_data[:, 2048:].reshape(-1, 32, 32, 1)
    train_data, test_data = np.concatenate((R, G, B), axis=3), np.concatenate((testR, testG, testB), axis=3)
    return train_data, train_labels, test_data, test_labels

In [ ]:
train_data, train_labels, test_data, test_labels = load_cifar10_img_form("datasets/cifar-10-batches")

In [ ]:
train_data, test_data = train_data.astype('float32') / 255.0, test_data.astype('float32') / 255.0
train_labels, test_labels = to_categorical(train_labels, 10), to_categorical(test_labels, 10)

train_data, val_data = train_data[:40000],  train_data[40000:]
train_labels, val_labels = train_labels[:40000], train_labels[40000:]

In [ ]:
def show_train_per_time(history, time_callback):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    times = time_callback.times
    
    plt.plot(times, acc, 'bo', label='Training acc')
    plt.plot(times, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xticks([times[0], times[-1]], visible=True, rotation='horizontal')
    plt.legend()
    plt.figure()

    plt.plot(times, loss, 'bo', label='Training loss')
    plt.plot(times, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xticks([times[0], times[-1]], visible=True, rotation='horizontal')
    plt.legend()
    plt.show()
    return

In [ ]:
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
        self.start_time = time.time()

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        elapsed_time = time.time() - self.start_time
        self.times.append(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.01, mode='min', patience=100)

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

model = models.Sequential()

# Block1
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=input_shape))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block2
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block3
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block4
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block5
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

# FC layers
model.add(layers.Flatten(name='flatten'))
model.add(layers.Dense(512, activation='relu', name='fc1'))
model.add(layers.Dense(512, activation='relu', name='fc2'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [ ]:
start_time = datetime.datetime.now().strftime('%B-%d %H:%M:%S')
time_callback = TimeHistory()
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels, batch_size=1000, epochs=1000, validation_data=(val_data, val_labels),
                    callbacks=[time_callback, early_stopping])

print('model fitting start:', start_time)
print('model fitting end and save:', datetime.datetime.now().strftime('%B-%d %H:%M:%S'))

In [ ]:
show_train_per_time(history, time_callback)

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
model.save('VGG16_full.h5')